In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
df = pd.read_csv("bike_trip_history_sample.csv")  # Replace with your file path

# --- 1. Handle Missing Values ---
# Drop rows with missing start or end time (critical)
df.dropna(subset=['start_time', 'end_time'], inplace=True)

# Fill missing gender with 'Unknown'
df['gender'] = df['gender'].fillna('Unknown')

# Fill missing birth_year with median
df['birth_year'] = df['birth_year'].fillna(df['birth_year'].median())

# --- 2. Convert start_time and end_time to datetime ---
df['start_time'] = pd.to_datetime(df['start_time'])
df['end_time'] = pd.to_datetime(df['end_time'])

# --- 3. Trip Duration (in minutes) ---
df['trip_duration'] = (df['end_time'] - df['start_time']).dt.total_seconds() / 60

# --- 4. Extract Time Features ---
df['start_hour'] = df['start_time'].dt.hour
df['day_of_week'] = df['start_time'].dt.day_name()
df['month'] = df['start_time'].dt.month

In [4]:
def duration_bin(minutes):
    if minutes <= 10:
        return "Short"
    elif minutes <= 30:
        return "Medium"
    else:
        return "Long"

df['duration_bin'] = df['trip_duration'].apply(duration_bin)

# --- 6. Calculate Age & Age Group ---
current_year = datetime.now().year
df['age'] = current_year - df['birth_year'].astype(int)

def age_group(age):
    if age < 25:
        return "Young"
    elif age <= 45:
        return "Adult"
    else:
        return "Senior"

df['age_group'] = df['age'].apply(age_group)

# --- 7. Peak Hour Feature (7-9AM and 5-7PM) ---
def is_peak_hour(hour):
    return hour in [7, 8, 17, 18]

df['peak_hour'] = df['start_hour'].apply(lambda h: 'Yes' if is_peak_hour(h) else 'No')

# --- 8. Drop Unnecessary Columns (Optional) ---
df.drop(columns=['trip_id', 'bike_id'], errors='ignore', inplace=True)

# --- 9. Save Cleaned Data ---
df.to_csv("cleaned_bike_trip_data.csv", index=False)

# Preview
print(df.head())

           start_time            end_time start_station end_station  \
0 2025-04-22 12:00:00 2025-04-22 12:10:00     Station B   Station F   
1 2025-04-02 04:00:00 2025-04-02 04:22:00     Station B   Station F   
2 2025-04-07 20:00:00 2025-04-07 20:55:00     Station B   Station F   
3 2025-04-18 13:00:00 2025-04-18 13:09:00     Station C   Station F   
4 2025-04-15 02:00:00 2025-04-15 02:51:00     Station C   Station F   

    user_type  birth_year  gender  trip_duration  start_hour day_of_week  \
0    Customer        1992  Female           10.0          12     Tuesday   
1  Subscriber        1984   Other           22.0           4   Wednesday   
2  Subscriber        1976    Male           55.0          20      Monday   
3    Customer        1992    Male            9.0          13      Friday   
4    Customer        1961   Other           51.0           2     Tuesday   

   month duration_bin  age age_group peak_hour  
0      4        Short   33     Adult        No  
1      4       Med